# Extracting Seattle-area landmass polygons

This is an optional notebook that generates the `artifacts/seattle_landmasses.geojson` file, which is already included in this repository by default. It depends on download the `processed_p.tar.bz2` file in the `data/landmasses` directory, with the URL for obtaining this listed below.

In [1]:
import warnings

import geopandas as gpd
from shapely.geometry import Polygon
from pyproj import Transformer

# Suppress warnings from Fiona regression
warnings.filterwarnings('ignore')

# Projections
WGS84 = 4326
WEB_MERCATOR = 3857
NAD83 = 26910

In [2]:
# Coastlines (actually landmasses) from https://tile.openstreetmap.org/processed_p.tar.bz2
!tar xf ../data/landmasses/processed_p.tar.bz2 -C ../data/landmasses/

In [3]:
# Extract (roughly) the Seattle area as a GeoDataFrame
transformer = Transformer.from_crs(WGS84, WEB_MERCATOR, always_xy=True)
seattle_bbox = (-122.5, 47.3, -122.1, 47.9)
southwest = transformer.transform(seattle_bbox[0], seattle_bbox[1])
northeast = transformer.transform(seattle_bbox[2], seattle_bbox[3])
web_mercator_bbox = (*southwest, *northeast)

landmass = gpd.read_file("../data/landmasses/processed_p.shp", bbox=web_mercator_bbox)
landmass.crs = WEB_MERCATOR

In [4]:
# Convert back to WGS84 and clip
landmass = landmass.to_crs(WGS84)
landmass = gpd.clip(landmass, Polygon.from_bounds(*seattle_bbox))
# Convert to NAD83 in meters
landmass = landmass.to_crs(NAD83)
landmass.head()

,error,tile_x,tile_y,geometry
0,0,63,259,"POLYGON ((545439.497 5241430.322, 545460.720 5..."
1,0,63,259,"POLYGON ((545375.419 5249733.263, 545390.144 5..."
2,0,63,260,"MULTIPOLYGON (((537423.924 5296919.809, 537432..."
3,0,63,260,"POLYGON ((542689.202 5252130.255, 537712.767 5..."
4,0,63,260,"POLYGON ((545273.876 5262870.340, 545274.699 5..."


In [5]:
# Merge into larger polygon
landmass = gpd.GeoDataFrame(geometry=gpd.GeoSeries(landmass.geometry.unary_union.geoms))
landmass.crs = NAD83

In [6]:
landmass.to_file("../artifacts/seattle_landmasses.geojson", driver="GPKG")